# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [1]:
!nvidia-smi

Mon Mar 15 16:07:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 207...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   40C    P8     4W / 215W |    278MiB /  7974MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7589780954269902944,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9163705885027782770
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13822638843212424826
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
from numba import cuda
cuda.is_available()

True

In [5]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.0167, 0.8647, 0.4834],
        [0.1561, 0.6817, 0.6721],
        [0.8204, 0.8983, 0.6740],
        [0.5128, 0.1414, 0.0441],
        [0.3337, 0.6559, 0.9986]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [6]:
x = np.random.rand(10000, 256)

In [7]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

264 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [8]:
x = torch.rand(10000, 256)

In [9]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

81.3 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 3) PyTorch on GPU via CUDA

In [10]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.2744, 0.3363, 0.1597, 0.0385, 0.8316],
        [0.7444, 0.2304, 0.3923, 0.1139, 0.6444],
        [0.8298, 0.7201, 0.3654, 0.1018, 0.8499],
        [0.1626, 0.4588, 0.8345, 0.3596, 0.8595],
        [0.2448, 0.3518, 0.5742, 0.1159, 0.2590]], device='cuda:0')
tensor([[0.2744, 0.3363, 0.1597, 0.0385, 0.8316],
        [0.7444, 0.2304, 0.3923, 0.1139, 0.6444],
        [0.8298, 0.7201, 0.3654, 0.1018, 0.8499],
        [0.1626, 0.4588, 0.8345, 0.3596, 0.8595],
        [0.2448, 0.3518, 0.5742, 0.1159, 0.2590]], dtype=torch.float64)


In [11]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

11.2 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [12]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [13]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.0620, 0.9917, 0.7712, 0.8506, 0.7601],
        [0.2086, 0.4004, 0.0151, 0.7307, 0.3851],
        [0.0313, 0.2739, 0.1238, 0.6561, 0.3260],
        [0.8486, 0.8079, 0.4257, 0.5574, 0.8970],
        [0.2714, 0.3015, 0.7008, 0.3282, 0.1581]], device='cuda:0')


In [14]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [15]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 1.1775e-03,  3.6999e-04,  1.3697e-04,  1.0167e-04,  1.2928e-04],
        [ 3.6999e-04,  9.8010e-04,  4.6505e-04,  4.8000e-05, -4.3864e-04],
        [ 1.3697e-04,  4.6505e-04,  1.2214e-03, -3.4245e-04, -1.7439e-04],
        [ 1.0167e-04,  4.8000e-05, -3.4245e-04,  9.3032e-04, -2.3356e-04],
        [ 1.2928e-04, -4.3864e-04, -1.7439e-04, -2.3356e-04,  6.2633e-04]],
       device='cuda:0')


In [16]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 1.1775e-03,  3.6999e-04,  1.3697e-04,  1.0167e-04,  1.2928e-04],
        [ 3.6999e-04,  9.8010e-04,  4.6505e-04,  4.8000e-05, -4.3864e-04],
        [ 1.3697e-04,  4.6505e-04,  1.2214e-03, -3.4245e-04, -1.7439e-04],
        [ 1.0167e-04,  4.8000e-05, -3.4245e-04,  9.3032e-04, -2.3356e-04],
        [ 1.2928e-04, -4.3864e-04, -1.7439e-04, -2.3356e-04,  6.2633e-04]],
       dtype=torch.float64)
